Importing libraries & functions

In [483]:
import sys
sys.executable


'c:\\Users\\USER\\Downloads\\document personelle\\document personelle\\projet_gestion_de_risque\\venv\\Scripts\\python.exe'

In [484]:
!{sys.executable} -m pip install pandas numpy scikit-learn


'c:\Users\USER\Downloads\document' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [485]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression

Importing dataset

In [486]:
from pathlib import Path

file_path = Path(
    "C:/Users/USER/Downloads/document personelle/document personelle/projet_gestion_de_risque/data/raw/base_SNI.xlsx"
)

df = pd.read_excel(file_path, sheet_name='base_sni')



Data preparation

In [487]:
# shows count of rows and columns
df.shape

(5752, 29)

In [488]:
#shows first few rows of the code
df.head()

,id_tiers,code_client,CRD,PNU,nb_pret,Maturite,Garantie_max,flag_lombard_decouvert,DPD 12M,Pays de domiciliation,...,Segment,date OBSERVATION,Age_relation,Date_naissance,Solde_moyen,Revenus_mensuels,Solde_arrete,Engagement,CRD/gar,CRD/eng
0,10231_2018,10231,3877.51,54792.50,1,-58,109585.0,1,0,France,...,Retail GOLD (08),2018-12-31,154,1963-01-14,NaN,NaN,-3877.510000,54792.50,0.035384,0.070767
1,10231_2019,10231,8087.98,45936.25,1,-59,0.0,1,0,France,...,Retail GOLD (08),2019-12-31,166,1963-01-14,NaN,NaN,-8115.410000,45936.25,1.000000,0.176070
2,10231_2020,10231,11702.36,45000.00,1,-60,0.0,1,0,France,...,Retail GOLD (08),2020-12-31,178,1963-01-15,NaN,NaN,-8115.410000,45000.00,1.000000,0.260052
3,10231_2021,10231,328.53,53085.75,1,-61,0.0,1,0,France,...,Retail GOLD (08),2021-12-31,190,1963-01-15,NaN,NaN,-328.530000,53085.75,1.000000,0.006189
4,10261_2018,10261,1277030.69,0.00,1,18,2475000.0,0,0,Morocco,...,"Private customer, universal (01)",2018-12-31,154,1978-01-10,NaN,NaN,78611.768604,2650000.00,0.515972,0.481898


In [489]:
#dropping customer id_tiers and code_client columns from the dataset
df=df.drop('id_tiers',axis=1)
df=df.drop('code_client',axis=1)
df.shape

(5752, 27)

In [490]:
# explore missing values
df.isna().sum()


CRD                             0
PNU                             0
nb_pret                         0
Maturite                        0
Garantie_max                    0
flag_lombard_decouvert          0
DPD 12M                         0
Pays de domiciliation        1753
Etat civil                   1753
Nationalité                  1753
Profession                   4588
Segmentation commercilale    1753
Type de revenu               1978
Catégorie sociale            1775
Code tribunal                5118
Code localité                1809
Interdiction cheque          5605
Segment                        64
date OBSERVATION                0
Age_relation                    0
Date_naissance               1753
Solde_moyen                  1547
Revenus_mensuels             2205
Solde_arrete                  302
Engagement                      5
CRD/gar                         0
CRD/eng                         5
dtype: int64

In [491]:
missing_rate = df.isnull().mean().sort_values(ascending=False)
missing_rate

Interdiction cheque          0.974444
Code tribunal                0.889777
Profession                   0.797636
Revenus_mensuels             0.383345
Type de revenu               0.343880
Code localité                0.314499
Catégorie sociale            0.308588
Etat civil                   0.304764
Date_naissance               0.304764
Nationalité                  0.304764
Pays de domiciliation        0.304764
Segmentation commercilale    0.304764
Solde_moyen                  0.268950
Solde_arrete                 0.052503
Segment                      0.011127
Engagement                   0.000869
CRD/eng                      0.000869
nb_pret                      0.000000
CRD                          0.000000
Maturite                     0.000000
Garantie_max                 0.000000
flag_lombard_decouvert       0.000000
DPD 12M                      0.000000
PNU                          0.000000
date OBSERVATION             0.000000
Age_relation                 0.000000
CRD/gar     

In [501]:
# create flag_default
df['flag_defaut'] = (df['DPD 12M'] >= 90).astype(int)

# Vérification simple
print(f"flag_defaut créé avec DPD >= 90")
print(f"Défauts (1) : {df['flag_defaut'].sum():,} clients")
print(f"Non-défauts (0) : {(df['flag_defaut'] == 0).sum():,} clients")
print(f"Taux de défaut : {df['flag_defaut'].mean()*100:.2f}%")

flag_defaut créé avec DPD >= 90
Défauts (1) : 120 clients
Non-défauts (0) : 5,632 clients
Taux de défaut : 2.09%


In [493]:
df = df.drop(columns=['Interdiction cheque', 'Code tribunal', 'Profession'])
print(f" Après suppression : {df.shape}")

 Après suppression : (5752, 25)


In [494]:
# numeric imputation
for col in ['Solde_arrete', 'Engagement', 'CRD/eng', 'Solde_moyen', 'Revenus_mensuels']:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].mean())

In [495]:
# categorical imputation
for col in ['Type de revenu', 'Pays de domiciliation', 'Etat civil', 'Nationalité', 
            'Code localité', 'Catégorie sociale', 'Segmentation commercilale']:
    if col in df.columns:
        df[col] = df[col].fillna('Inconnu')

if 'Segment' in df.columns:
    df['Segment'] = df['Segment'].fillna(df['Segment'].mode()[0])

In [496]:
#create age variable from date of birth
if 'Date_naissance' in df.columns:
    try:
        # Convertir en datetime
        df['Date_naissance'] = pd.to_datetime(df['Date_naissance'], errors='coerce')
        
        # Date de référence
        if 'date OBSERVATION' in df.columns:
            df['date OBSERVATION'] = pd.to_datetime(df['date OBSERVATION'], errors='coerce')
            date_ref = df['date OBSERVATION'].mode()[0]
        else:
            from datetime import datetime
            date_ref = datetime.now()
        
        # Calculer l'âge médian pour imputation
        ages_connus = ((date_ref - df['Date_naissance']).dt.days / 365.25).dropna()
        age_median = ages_connus.median()
        
        # Imputer les dates manquantes
        date_naissance_mediane = date_ref - pd.Timedelta(days=int(age_median * 365.25))
        df['Date_naissance'].fillna(date_naissance_mediane, inplace=True)
        
        # Créer la variable Age
        df['Age'] = ((date_ref - df['Date_naissance']).dt.days / 365.25).round(0)
        
        # Nettoyer les âges aberrants
        df['Age'] = df['Age'].clip(lower=18, upper=100)
        
        print(f"   Age créé (médiane = {age_median:.1f} ans)")
        print(f"      → Min: {df['Age'].min():.0f} | Max: {df['Age'].max():.0f} | Moyenne: {df['Age'].mean():.1f}")
        
    except Exception as e:
        print(f"  Erreur création Age : {e}")




   Age créé (médiane = 38.0 ans)
      → Min: 18 | Max: 88 | Moyenne: 39.7


C:\Users\USER\AppData\Local\Temp\ipykernel_12336\765326834.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Date_naissance'].fillna(date_naissance_mediane, inplace=True)


In [497]:
# Sauvegarder
df.to_excel('data/processed/base_propre.xlsx', index=False)
df.to_csv('data/processed/base_propre.csv', index=False)

print(f" Terminé ! {df.shape} | Manquants: {df.isnull().sum().sum()}")

 Terminé ! (5752, 26) | Manquants: 0


In [498]:
## explore missing values after imputation
df.isna().sum()

CRD                          0
PNU                          0
nb_pret                      0
Maturite                     0
Garantie_max                 0
flag_lombard_decouvert       0
DPD 12M                      0
Pays de domiciliation        0
Etat civil                   0
Nationalité                  0
Segmentation commercilale    0
Type de revenu               0
Catégorie sociale            0
Code localité                0
Segment                      0
date OBSERVATION             0
Age_relation                 0
Date_naissance               0
Solde_moyen                  0
Revenus_mensuels             0
Solde_arrete                 0
Engagement                   0
CRD/gar                      0
CRD/eng                      0
flag_defaut                  0
Age                          0
dtype: int64

Train Test Split

In [499]:
y = df.iloc[:, 25].values
X = df.iloc[:, 1:26].values

In [500]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

NameError: name 'X_train' is not defined

Risk Model building
